In [1]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [10]:
url = 'https://www.dbpia.co.kr/search/topSearch?searchOption=all&query=%EB%94%A5%EB%9F%AC%EB%8B%9D'

res = []

driver = webdriver.Chrome() # 크롬 드라이버 생성
driver.get(url) # 사이트 접속하기

checkbox = driver.find_element(By.ID, 'domesticRecord_064001') # KCI 등재 체크박스

if not checkbox.is_selected(): # 만약 KCI 등재 체크박스가 체크되어 있지 않다면 체크
    checkbox.click()

driver.implicitly_wait(10) # 페이지 로딩이 완료될 때까지 대기

cnt = 0 
page = []

for n in range(1,10): # KCI에 등재된 논문은 총 1943건(한 페이지당 20건씩 있음)
    driver.execute_script("onclick=setPageNum({})".format(n))
    # 자바스크립트로 페이지가 이동되기 때문에 해당 코드를 실행
    time.sleep(10) # 페이지가 완전히 로드될 때까지 10초 대기
    html = driver.page_source 

    soup = BeautifulSoup(html, 'lxml') # 페이지 소스를 lxml로 변환
    # lxml : String형식의 HTML문서 -> 살아있는 HTML문서

    for_thesis = soup.find('section',attrs={"id":"searchResultList"}) # 논문 목록

    for i in for_thesis.find_all('article',attrs={'class':'thesis__summary'}):
        title = i.find('h2').get_text() # 제목 
        date = i.find('span',attrs={'class':'thesis__item'}).get_text() # 날짜
        abstract = i.find('span',attrs={'class':'thesis__abstract'}) # abstract 
        abstract2 = [abstract.get_text() if abstract != None else ''][0] 
        # 만약 abstract가 존재하지 않는다면 ''으로 대신함
        if date[:4] in [str(j) for j in range(2014,2024)]: # 2014~2023년에 해당되는 논문만(1430건)
            res += [[title, date, abstract2]] # [제목, 날짜, abstract 내용]을 리스트에 저장함
            cnt += 1

driver.close()

In [11]:
# 총 1430건의 논문 수집 성공
len(res)

35

In [12]:
df = pd.DataFrame(res, columns=['제목','날짜','abstract'])
df.info()

df.to_csv('dbpia2014_2023_미술_치료.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   제목        35 non-null     object
 1   날짜        35 non-null     object
 2   abstract  35 non-null     object
dtypes: object(3)
memory usage: 972.0+ bytes


In [13]:
dff = pd.read_csv('dbpia2014_2023_미술_치료.csv',encoding='utf-8')
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  35 non-null     int64  
 1   제목          35 non-null     object 
 2   날짜          35 non-null     float64
 3   abstract    23 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.2+ KB


In [14]:
dff.head()

,Unnamed: 0,제목,날짜,abstract
0,0,Sentinel-2 위성과 국토위성에서의 딥러닝 기반 초해상화 기법 비교 연구,2023.12,딥러닝 기반 초해상화 모델은 저해상도 영상을 효과적으로 고해상도로 변환하여 보다 선...
1,1,시뮬레이션 데이터로 학습된 딥러닝 모델을 사용한 항공 라이다 데이터의 건물 모델링,2023.12,이를 위해 딥러닝 기술을 사용하여 건물 모델링의 중요한 과정인 지붕면 분할을 수행하...
2,2,위상잠금 적외선열화상의 가우시안 푸리에 변환 및 딥러닝을 활용한 해상도 향상 연구,2023.12,"또한, 노이즈가 존재하는 이미지의 해상도를 향상하기 위해 딥러닝 기법 중 초고해상도..."
3,3,딥러닝 기반 초해상화 모델을 이용한 드론 사진의 해상도 향상 연구,2023.12,"고해상 위성 영상 및 항공 영상을 이용할 수 있으나, 경제적 부담으로 인해 지속적인..."
4,4,공간 질의응답 시스템에서의 개체링킹과 딥러닝 모델을 활용한 멘션 탐지,2023.12,"이를 위해 딥러닝 모델인 BERT, RoBERTa 그리고 ChatGPT를 사용하여 ..."


In [ ]:
#-*-coding:utf-8-*-
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
import re
import csv
import datetime

searchQ = "deeplearning"
startYear = 2014
endYear = 2024

print("start crawling..")

path = 'chromedriver'
driver = webdriver.Chrome(path)
xpath = driver.find_element_by_xpath

#스타트업 검색
driver.get('https://www.dbpia.co.kr/')
keyword = driver.find_element_by_id('keyword')
keyword.clear()
keyword.send_keys(searchQ)
serach_click_btn = xpath("//*[@id='header']/div[4]/div[2]/div[1]/div[1]/a")
driver.execute_script("arguments[0].click();",serach_click_btn)

#날짜 지정 및 체크박스 설정
xpath("//*[@id='dev_sartYY']").send_keys(startYear)
xpath("//*[@id='dev_endYY']").send_keys(endYear)
click_btn = xpath("//*[@id='sidebar']/form/div[2]/div/div[1]/ul/li[4]/p/button")
driver.execute_script("arguments[0].click();",click_btn)

for i in range(1):
    time.sleep(0.5)
    try:
        btn = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='pub_check_sort3_0']")))
        driver.execute_script("arguments[0].click()",btn)
        btn2 = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='pub_check_sort3_1']")))
        driver.execute_script("arguments[0].click()",btn2)
    except:
        print('retry')
        time.sleep(1)

# base_url = 'https://www.dbpia.co.kr/search/topSearch#none'
# test_list = []
# driver.get(base_url)

# 더보기
wCount = 0
while(True):
    time.sleep(1)
    try:
        more = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='contents']/div[2]/div[3]/div[3]/div[3]/div/a")))
        driver.execute_script("arguments[0].click()",more)
    except:
        print('retry')
        break
    wCount += 1
    print(" + page [{}]".format(wCount))


items_source = driver.page_source
soup = BeautifulSoup(items_source, 'html.parser')


items = soup.find('div','searchListArea').find('div','listBody').find('ul').find_all('li', 'item')

# 논문제목, 저자, 퍼블리셔, 저널명,볼륨,날짜,초록
titleL = []
authorL = []
# authorsL = []
publisherL = []
journalL = []
volumeL = []
dateL = []
abstractL = []
tLen = len(items)
print("start parsing")

iCount = 0
for item in items :
    iCount += 1
    if iCount % 20 == 0:
        print(" parsing.. [{}/{}]".format(iCount, tLen))

    title = ''
    try : title = item.find('div','titWrap').find('a').text
    except : title = ''

    author = ''
    try : author = item.find('li','author').text
    except : author = ''

    authors = ''
    try : authors = item.find('li','author').find('input')['value']
    except : authors = ''

    publisher = ''
    try : publisher = item.find('li','publisher').text
    except : publisher = ''

    journal = ''
    try : journal = item.find('li','journal').text
    except : journal = ''

    volume = ''
    try : volume = item.find('li','volume').text
    except : volume = ''

    date = ''
    try : date = item.find('li','date').text
    except : date = ''

    abstract = ''
    baseDetailUrl = "https://www.dbpia.co.kr"
    pUrl = ''
    try : pUrl = item.find('div','titWrap').find('a')['href']
    except : pUrl = ''
    if (pUrl != ''):
        pUrl = baseDetailUrl + pUrl
        driver.get(pUrl)
        try : driver.find_element_by_xpath('//*[@id="#pub_modalOrganPop"]').click()
        except : pass
        time.sleep(0.1)
        try : driver.find_element_by_xpath('//*[@id="#pub_modalLoginPop"]').click()
        except : pass

        try :
            driver.find_element_by_xpath('//*[@id="pub_abstract"]/div[2]/div/div[1]/div[2]/a').click()
            eachPage = driver.page_source
            ePsoup = BeautifulSoup(eachPage, 'html.parser')
            abstract = ePsoup.find('div','abstFull').find('p','article').text
        except : abstract = ''

    titleL.append(title)
    authorL.append(author)
    # authorsL.append(authors)
    publisherL.append(publisher)
    journalL.append(journal)
    volumeL.append(volume)
    dateL.append(date)
    abstractL.append(abstract)

print("date to .csv file")

resultDict = dict(title = titleL,
              author = authorL,
              publisher = publisherL,
              journal = journalL,
              volume = volumeL,
              date = dateL,
              abstract = abstractL)

fName = "dbpia_{}_{}_{}.csv".format(searchQ, startYear, endYear)

DB = pd.DataFrame(resultDict)
DB.to_csv(fName)